In [1]:
import numpy 


In [3]:
out = numpy.load("best_model.npz")

In [7]:
out.files

['b', 'W']

In [174]:
out['W']

(3072, 2)

In [170]:
a=numpy.random.randn(n_in,)
#a.dtype="float32"


In [173]:
numpy.sum(a>0)

1522

In [227]:
import numpy
import theano
from theano import tensor

batch_size = 128

# Size of the data
n_in = 32 * 32 * 3 
# Number of classes
n_out = 2

x = tensor.matrix('x')
W = theano.shared(value=numpy.random.randn(n_in,n_out),
                  name='W',
                  borrow=True)
b = theano.shared(value=numpy.random.randn(n_out,),
                  name='b',
                  borrow=True)
                  
p_y_given_x = tensor.nnet.softmax(tensor.dot(x, W) + b)
y_pred = tensor.argmax(p_y_given_x, axis=1)

y = tensor.lvector('y')
log_prob = tensor.log(p_y_given_x)
log_likelihood = log_prob[tensor.arange(y.shape[0]), y]
loss = - log_likelihood.mean()

g_W, g_b = theano.grad(cost=loss, wrt=[W, b])

learning_rate = numpy.float32(0.13)
new_W = W - learning_rate * g_W
new_b = b - learning_rate * g_b

train_model = theano.function(inputs=[x, y],
                              outputs=loss,
                              updates=[(W, new_W),
                                       (b, new_b)])
                                       

misclass_nb = tensor.neq(y_pred, y)
misclass_rate = misclass_nb.mean()

test_model = theano.function(inputs=[x, y],
                             outputs=misclass_rate)

In [228]:
import os
import timeit

#os.environ['FUEL_DATA_PATH'] = os.path.abspath('./fuel_data')

# Let's load and process the dataset
from fuel.datasets.dogs_vs_cats import DogsVsCats
from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
from fuel.transformers.image import RandomFixedSizeCrop
from fuel.transformers.image import MinimumImageDimensions
from fuel.transformers import Flatten
from fuel.transformers import ScaleAndShift


train_set = DogsVsCats(('train',), subset=slice(0, 20000))
valid_set = DogsVsCats(('train',), subset=slice(20000, 25000))
test_set = DogsVsCats(('test',))

batch_size = 128
n_train_batches = train_set.num_examples // batch_size

#################################################################################
# Train Stream
# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream instead of DataStream.default_stream constructor we return
# our images exactly as is.
stream = DataStream(
    train_set,
    iteration_scheme=ShuffledScheme(train_set.num_examples, batch_size)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to upscale images to at least (512 x 512)

upscale_stream = MinimumImageDimensions(stream, (256,256),which_sources=('image_features',))

# Take random crops of (32 x 32) from each image
cropped_stream = RandomFixedSizeCrop(
    upscale_stream, (32, 32), which_sources=('image_features',))

# Convert images to [0,1] scale
default_cropped_stream = ScaleAndShift(cropped_stream,  1.0/(255.0), 0., which_sources=('image_features',))
# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
train_stream = Flatten(
    default_cropped_stream, which_sources=('image_features',))
    
######################################################################################
# Valid Stream

# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream instead of DataStream.default_stream constructor we return
# our images exactly as is.
stream = DataStream(
    valid_set,
    iteration_scheme=ShuffledScheme(valid_set.num_examples, batch_size)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to upscale images to at least (512,512)

valid_upscale_stream = MinimumImageDimensions(stream, (256,256),which_sources=('image_features',))

# Take random crops of (32 x 32) from each image
valid_cropped_stream = RandomFixedSizeCrop(
    valid_upscale_stream, (32, 32), which_sources=('image_features',))

# Convert images to [0,1] scale
valid_default_cropped_stream = ScaleAndShift(valid_cropped_stream,  1.0/(255.0), 0., which_sources=('image_features',))
# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
valid_stream = Flatten(
    valid_default_cropped_stream, which_sources=('image_features',))
    
##########################################################################################
# Test Stream 

# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream instead of DataStream.default_stream constructor we return
# our images exactly as is.
stream = DataStream(
    test_set,
    iteration_scheme=ShuffledScheme(test_set.num_examples, batch_size)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to upscale images to at least (512,512)

test_upscale_stream = MinimumImageDimensions(stream, (256,256),which_sources=('image_features',))

# Take random crops of (32 x 32) from each image
test_cropped_stream = RandomFixedSizeCrop(
    test_upscale_stream, (32, 32), which_sources=('image_features',))

# Convert images to [0,1] scale
test_default_cropped_stream = ScaleAndShift(test_cropped_stream,  1.0/(255.0), 0., which_sources=('image_features',))
# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
test_stream = Flatten(
    test_default_cropped_stream, which_sources=('image_features',))
    
####################################################################################################



In [229]:
dotprodlist = []
for test_xi, test_yi in valid_stream.get_epoch_iterator():
    for i in range(test_xi.shape[0]):
        dotprodlist.append(numpy.dot(test_xi[i],W.eval()[:,1])+b.eval()[1])
    

In [230]:
numpy.mean(dotprodlist)

2.7900141255718038

In [231]:
dotprodlist

[2.0643844674550023,
 8.7081844283546666,
 5.6620300752764576,
 5.3498462685436463,
 7.61580077376109,
 1.5688769585216698,
 -1.1820548164320079,
 3.6131450321640619,
 -10.097973931690582,
 -2.3857092450671233,
 1.5579741993480731,
 -0.87294393713853125,
 -5.5173199918777742,
 12.403675884888683,
 8.5746514549089419,
 -0.14596271834557917,
 0.75806167431879534,
 -0.6789916969850267,
 10.572148945941244,
 3.3458543417467679,
 7.8316786579435123,
 1.4005064071550948,
 3.3299513265396525,
 -5.0534660943561365,
 2.4458335709416792,
 3.469212616918365,
 10.68079940703743,
 -2.3034602641099911,
 1.1317090695904024,
 2.5826567567284497,
 -8.9466070379250091,
 -0.36770679280339968,
 12.016550138840515,
 1.559174068449191,
 -0.28374147151612017,
 1.7428994432314397,
 10.691108723775891,
 3.3377440774718812,
 3.3704492317280836,
 0.84098567253122458,
 -2.8997149790946084,
 11.795040017113042,
 10.822269867923755,
 -3.3438054286516392,
 1.6206073228857572,
 11.01095363117852,
 13.890533172490695,

In [126]:
validation_losses = []
for valid_xi, valid_yi in valid_stream.get_epoch_iterator():
    validation_losses.append(test_model(valid_xi, valid_yi.flatten()))



In [127]:
validation_losses

[array(0.4921875),
 array(0.4765625),
 array(0.4921875),
 array(0.6015625),
 array(0.53125),
 array(0.4921875),
 array(0.4375),
 array(0.453125),
 array(0.5390625),
 array(0.578125),
 array(0.40625),
 array(0.578125),
 array(0.4921875),
 array(0.4765625),
 array(0.5234375),
 array(0.46875),
 array(0.515625),
 array(0.421875),
 array(0.5546875),
 array(0.515625),
 array(0.5234375),
 array(0.546875),
 array(0.4609375),
 array(0.4765625),
 array(0.5546875),
 array(0.53125),
 array(0.5234375),
 array(0.46875),
 array(0.4765625),
 array(0.4609375),
 array(0.5390625),
 array(0.53125),
 array(0.4375),
 array(0.5390625),
 array(0.5),
 array(0.5078125),
 array(0.5390625),
 array(0.484375),
 array(0.5625),
 array(0.25)]

In [24]:
sum(validation_losses)/len(validation_losses)

0.47812500000000002

In [124]:
test_losses = []
for test_xi, test_yi in test_stream.get_epoch_iterator():
    test_losses.append(test_model(test_xi, test_yi.flatten()))




In [125]:
test_losses

[array(0.015625),
 array(0.0234375),
 array(0.03125),
 array(0.0390625),
 array(0.0),
 array(0.015625),
 array(0.0078125),
 array(0.0234375),
 array(0.015625),
 array(0.015625),
 array(0.03125),
 array(0.03125),
 array(0.0078125),
 array(0.0078125),
 array(0.0078125),
 array(0.0078125),
 array(0.0234375),
 array(0.046875),
 array(0.015625),
 array(0.03125),
 array(0.0),
 array(0.0234375),
 array(0.0078125),
 array(0.0),
 array(0.0234375),
 array(0.0234375),
 array(0.0078125),
 array(0.0390625),
 array(0.015625),
 array(0.015625),
 array(0.03125),
 array(0.0234375),
 array(0.0234375),
 array(0.0),
 array(0.015625),
 array(0.0078125),
 array(0.0234375),
 array(0.0078125),
 array(0.0078125),
 array(0.0234375),
 array(0.046875),
 array(0.015625),
 array(0.03125),
 array(0.015625),
 array(0.015625),
 array(0.0234375),
 array(0.0234375),
 array(0.015625),
 array(0.0),
 array(0.0078125),
 array(0.0234375),
 array(0.03125),
 array(0.03125),
 array(0.0),
 array(0.046875),
 array(0.046875),
 arr

In [232]:
output_model = theano.function(inputs=[x],
                             outputs=y_pred)

In [233]:
test_predicted = []
for test_xi, test_yi in test_stream.get_epoch_iterator():
    #test_losses.append(output_model(test_xi, test_yi.flatten()))
    test_predicted.append(output_model(test_xi).flatten())



In [234]:
test_predicted 

[array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]),
 array([0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0]),
 array([1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
        1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,

In [137]:
output_model2= theano.function(inputs=[x],
                             outputs=p_y_given_x)
testargmax = tensor.argmax(p_y_given_x,axis=1)
test_y_pred = []
temp1list = []
temp2list = []
temp3list = []
for test_xi, test_yi in test_stream.get_epoch_iterator():
    #test_losses.append(output_model(test_xi, test_yi.flatten()))
    temp1list.append(output_model2(test_xi))
    temp2list.append(output_model(test_xi))
    temp3list.append(testargmax.eval({x:test_xi}))
    #test_p_y_given_x.append(output_model2(test_xi))

#test_p_y_given_x

In [138]:
temp1list

[array([[  4.98210982e-37,   1.00000000e+00],
        [  3.12958658e-52,   1.00000000e+00],
        [  9.16541932e-30,   1.00000000e+00],
        [  1.43345849e-13,   1.00000000e+00],
        [  6.06762839e-44,   1.00000000e+00],
        [  7.79197950e-31,   1.00000000e+00],
        [  1.94818498e-02,   9.80518150e-01],
        [  2.86904551e-30,   1.00000000e+00],
        [  3.99046787e-19,   1.00000000e+00],
        [  3.39086538e-31,   1.00000000e+00],
        [  7.40160438e-21,   1.00000000e+00],
        [  3.56730540e-34,   1.00000000e+00],
        [  5.76464335e-18,   1.00000000e+00],
        [  6.22432828e-34,   1.00000000e+00],
        [  5.84589439e-47,   1.00000000e+00],
        [  2.43619371e-02,   9.75638063e-01],
        [  5.25927301e-27,   1.00000000e+00],
        [  1.17200836e-34,   1.00000000e+00],
        [  4.89631800e-13,   1.00000000e+00],
        [  3.14395763e-18,   1.00000000e+00],
        [  3.42758012e-22,   1.00000000e+00],
        [  4.58203372e-23,   1.000

In [139]:
temp2list

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [140]:
temp3list

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [75]:

testargmax = tensor.argmax(p_y_given_x,axis=1)

In [90]:
testargmax.eval({x:test_xi})

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [59]:
p_y_given_x

Softmax.0

In [73]:
temp2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [92]:
b.eval()

array([ 1.14754855,  1.99701942])

In [117]:
test_xi[1,:]

array([ 0.23529412,  0.25490196,  0.2       , ...,  0.43137255,
        0.31764706,  0.20392157])

In [119]:
test_set.num_examples

12500

In [132]:
#check balance of categories
yi_class = []
for valid_xi, valid_yi in valid_stream.get_epoch_iterator():
    yi_class.append(sum(valid_yi.flatten()))

In [134]:
sum(yi_class)

2470

In [166]:
# only feed dogs to model
output_model2= theano.function(inputs=[x],
                             outputs=p_y_given_x)
testargmax = tensor.argmax(p_y_given_x,axis=1)
test_y_pred = []
temp1list = []
temp2list = []
temp3list = []
for test_xi, test_yi in valid_stream.get_epoch_iterator():
    index = [test_yi.flatten() ==1]
    test_xi = test_xi[index]
    #test_losses.append(output_model(test_xi, test_yi.flatten()))
    temp1list.append(output_model2(test_xi))
    temp2list.append(output_model(test_xi))
    temp3list.append(testargmax.eval({x:test_xi}))
    #test_p_y_given_x.append(output_model2(test_xi))

#test_p_y_given_x

In [142]:
test_xi, test_yi = next(valid_stream.get_epoch_iterator())

In [149]:
index = [test_yi.flatten() ==1]

In [158]:
test_xi[index].shape

(62, 3072)

In [165]:
numpy.sum([term.astype(int) for term in index])

62

In [167]:
temp1list

[array([[  1.54703885e-16,   1.00000000e+00],
        [  1.54789821e-20,   1.00000000e+00],
        [  3.13056920e-27,   1.00000000e+00],
        [  3.28805083e-26,   1.00000000e+00],
        [  9.46827011e-45,   1.00000000e+00],
        [  8.78596328e-17,   1.00000000e+00],
        [  1.81331561e-23,   1.00000000e+00],
        [  9.63172455e-47,   1.00000000e+00],
        [  1.46015169e-34,   1.00000000e+00],
        [  1.39608288e-46,   1.00000000e+00],
        [  1.15542315e-13,   1.00000000e+00],
        [  9.41560358e-55,   1.00000000e+00],
        [  3.89316863e-34,   1.00000000e+00],
        [  2.10535901e-21,   1.00000000e+00],
        [  7.38311023e-43,   1.00000000e+00],
        [  1.98974190e-40,   1.00000000e+00],
        [  3.70825135e-05,   9.99962917e-01],
        [  2.05890711e-29,   1.00000000e+00],
        [  4.52524565e-33,   1.00000000e+00],
        [  1.22699063e-25,   1.00000000e+00],
        [  2.63270577e-38,   1.00000000e+00],
        [  2.08603747e-23,   1.000

In [168]:
temp2list

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [169]:
temp3list

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [181]:
dotprodlist = []
for test_xi, test_yi in valid_stream.get_epoch_iterator():
    for i in range(test_xi.shape[0]):
        dotprodlist.append(numpy.dot(test_xi[i],W.eval()[:,1])+b.eval()[1])
    

In [198]:
numpy.sum(test_xi[0]>0.2)

2829

In [216]:
b.eval()

array([ 1.53634076,  0.12694045])

In [202]:
numpy.sum(W.eval()[:,1]< 2 )

3008

In [180]:
W.eval()[:,1]

array([ 1.10773513,  2.5718099 ,  0.80417096, ...,  0.33627209,
       -0.12650122, -1.19288292])

In [184]:
dotprodlist

[47.089409834850173,
 75.988822973050887,
 70.445078001586623,
 27.771780128556635,
 85.791338127877012,
 16.035408398378976,
 42.273427619935497,
 31.668845269664647,
 28.030507087140766,
 65.03791039171027,
 91.278343686561115,
 40.425452666680961,
 50.996422538959301,
 34.550051968729647,
 52.191293868254519,
 54.499005614492681,
 28.323456794146267,
 21.267825362986283,
 101.57031517456166,
 24.345211262902598,
 54.047666518658595,
 78.524260092638215,
 31.345225073829599,
 16.263324331558643,
 40.922715763424314,
 35.947409021775293,
 29.018432708390343,
 34.056445938021199,
 66.606014454483272,
 71.24886263544677,
 59.511613193335485,
 28.697830611474913,
 93.026861790187851,
 59.511716901749146,
 47.758906610991254,
 95.93536762267864,
 15.252222686179421,
 80.678691435810634,
 30.175728281230271,
 48.492703480166753,
 61.518029485241293,
 65.866426010347695,
 40.785824211304764,
 51.851053069236094,
 50.710369086714771,
 35.933508444581037,
 31.820956549556222,
 71.255011912459